In [2]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  WORKER_COUNT  PREEMPTIBLE_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-ed47  GCE       4                                       RUNNING  us-central1-a


In [3]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [4]:
# !pip3 install ipykernel --upgrade
# ! python3 -m ipykernel install --user


In [5]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
hash("kaka")

4803017923801119554

In [7]:
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Jan  4 17:11 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [8]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [9]:
spark

In [10]:
bucket_name = '316431873' 
client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    print(b.name)

graphframes.sh
indexes/index2.pkl
indexses/index.pkl
indexses/index_anc.pkl
pagerank/pr/
pagerank/pr/_SUCCESS
pagerank/pr/part-00000-29992333-94e8-4c40-83cf-cede9ba3581a-c000.csv.gz
postings_gcp/0_000.bin
postings_gcp/0_001.bin
postings_gcp/0_002.bin
postings_gcp/0_003.bin
postings_gcp/0_004.bin
postings_gcp/0_005.bin
postings_gcp/0_006.bin
postings_gcp/0_007.bin
postings_gcp/0_008.bin
postings_gcp/0_009.bin
postings_gcp/0_010.bin
postings_gcp/0_011.bin
postings_gcp/0_012.bin
postings_gcp/0_013.bin
postings_gcp/0_014.bin
postings_gcp/0_015.bin
postings_gcp/0_016.bin
postings_gcp/0_017.bin
postings_gcp/0_018.bin
postings_gcp/0_019.bin
postings_gcp/0_020.bin
postings_gcp/0_021.bin
postings_gcp/0_022.bin
postings_gcp/0_023.bin
postings_gcp/0_024.bin
postings_gcp/0_025.bin
postings_gcp/0_026.bin
postings_gcp/0_027.bin
postings_gcp/0_028.bin
postings_gcp/0_posting_locs.pickle
postings_gcp/100_000.bin
postings_gcp/100_001.bin
postings_gcp/100_002.bin
postings_gcp/100_003.bin
postings_gcp/100

In [11]:
full_path = "gs://wikidata_preprocessed/*"
parquetFile = spark.read.parquet(full_path)


In [12]:
parquetFile.show()

+-------+--------------------+--------------------+--------------------+
|     id|               title|                text|         anchor_text|
+-------+--------------------+--------------------+--------------------+
|4045403|Foster Air Force ...|'''Foster Air For...|[{1176764, Tactic...|
|4045413|     Torino Palavela|'''Palavela''', f...|[{77743, 2006 Win...|
|4045419|   Mad About the Boy|"'''Mad About the...|[{34028256, Joyce...|
|4045426|       Shayne Breuer|'''Shayne Breuer'...|[{1838386, Woodvi...|
|4045432|         Parantaka I|'''Parantaka Chol...|[{1511716, Aditya...|
|4045456|Arundel (UK Parli...|'''Arundel''' was...|[{4665376, Arunde...|
|4045466|     Andrew Martinez|'''Luis Andrew Ma...|[{4860, Berkeley,...|
|4045471|    Vancouver VooDoo|The '''Vancouver ...|[{32706, Vancouve...|
|4045479|     Invisible plane|The '''Invisible ...|[{2260539, Ross A...|
|4045516|    Shopping channel|'''Shopping chann...|[{592899, special...|
|4045519|      Turgay (river)|The '''Turgay''' ...|

In [13]:
# Count number of wiki pages
parquetFile.count()

6348910

In [14]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [15]:
# # # adding our python module to the cluster
# sc.addFile("/home/dataproc/inverted_anc2.py")
# sys.path.insert(0,SparkFiles.getRootDirectory())

In [16]:
# # # adding our python module to the cluster
# sc.addFile("/home/dataproc/inverted_title.py")
# sys.path.insert(0,SparkFiles.getRootDirectory())

In [17]:

# # adding our python module to the cluster
# sc.addFile("/home/dataproc/inverted_index_colab.py")
# sys.path.insert(0,SparkFiles.getRootDirectory())


In [18]:
# from inverted_anc2 import *

In [19]:
from inverted_index_gcp import *

In [20]:
# from inverted_title import *

# get only the first 1000 rows

In [19]:
doc_text_pairs = parquetFile.limit(2).select("text", "id").rdd
doc_title_pairs = parquetFile.select("title", "id").rdd
doc_anchor_text_pairs = parquetFile.select("id", "anchor_text").rdd
parquetFile.createOrReplaceTempView("df")

text_doc_pairs = parquetFile.select("id","text").rdd



In [21]:

def get_title_by_id(id):
  sqlDF = spark.sql("SELECT title FROM df where id in ({})".format(id))
  return sqlDF.collect()[0][0]
  


get_title_by_id(12)

create a sample of each data segment (text,title,anchor)

at the following format: 

    (text/title/anchor , wiki_id)

functions from assigment 3 :

word count 

reduce_word_counts

calculate_df

In [5]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

all_stopwords = english_stopwords.union(corpus_stopwords)

def word_count(text, id):
  ''' Count the frequency of each word in `text` (tf) that is not included in 
  `all_stopwords` and return entries that will go into our posting lists. 
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs 
      for example: [("Anarchism", (12, 5)), ...]
  '''
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  frq = {}
  ans = []
  for word in tokens: 
    if word not in all_stopwords: 
       if word not in frq.keys():
             frq[word] = 1
       else:
             frq[word] +=1 
  for key, val in frq.items():
      curr = (key, (id,val))
      ans.append(curr)
  return ans




def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples 
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  ans = sorted(unsorted_pl, key= lambda x : x[0])
  return ans


def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  return postings.map(lambda y : (y[0], len(y[1])))
  

NameError: name 'stopwords' is not defined

In [2]:
word_counts = doc_anchor_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))


NameError: name 'doc_anchor_text_pairs' is not defined

two kind of function :
create posting file function for each part
bucket naming policy:

Hash finction - token2bucket_id, token2bucket_id_title,token2bucket_id_anc - hash functions with the following bin naming policy:

     [0,124] - text buckets 

     [125,249] -  anc buckets

     [250 ,379] -  title buckets
2.Posting list functions - a function that take a token and alocate the file in acordnce to the bin name , and then write the posting list into the bin based in the built in inverted index

        partition_postings_and_write_text  , partition_postings_and_write_title, 
        artition_postings_and_write_anc 

In [26]:
NUM_BUCKETS = 250
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def token2bucket_id_anc(token):
  return (int(_hash(token),16) % NUM_BUCKETS) + 125

def token2bucket_id_title(token):
  return (int(_hash(token),16) % NUM_BUCKETS) + 125*2  
 
def partition_postings_and_write_text(postings):
  ans =postings.map(lambda tup:  (token2bucket_id(tup[0]) , tup))
  return   ans.groupByKey().map(lambda tup:  InvertedIndex.write_a_posting_list(tup, bucket_name))

def partition_postings_and_write_title(postings):
  ans =postings.map(lambda tup:  (token2bucket_id_title(tup[0]) , tup))
  return   ans.groupByKey().map(lambda tup:  InvertedIndex_title.write_a_posting_list(tup, bucket_name))



In [32]:
# create a flat map so we would be able to group by key: 

In [29]:
word_counts = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))
anchor_word_counts = doc_anchor_text_pairs.flatMap(lambda x: word_count(x[1], x[0]))


KeyboardInterrupt: 

In [34]:
# create a flat posting rdd for each part

In [35]:
postings = word_counts.groupByKey().mapValues(reduce_word_counts)
postings_title = title_word_counts.groupByKey().mapValues(reduce_word_counts)


###  frequency and filter


In [36]:
  # filter a a posting list the does not meet the 10+ times freq (except from the title)

In [17]:
postings_filtered = postings.filter(lambda x: len(x[1])>50)
postings_title_filtered = postings_title.filter(lambda x: len(x[1]) >0)


  # create posting list at with bins 

posting_locs_list_title = partition_postings_and_write_title(postings_title_filtered).collect()



NameError: name 'postings' is not defined

In [ ]:
posting_locs_list = partition_postings_and_write_text(postings_filtered).collect()





In [35]:
posting_locs_list

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123]

In [39]:
# create (term, df) dict 


w2df_text = calculate_df(postings_filtered)
w2df_dict = w2df_text.collectAsMap()

w2df_title = calculate_df(postings_title_filtered)
w2df_dict_title = w2df_title.collectAsMap()



In [37]:
postings_anchor_filtered

NameError: name 'postings_anchor_filtered' is not defined

In [40]:
_ = partition_postings_and_write_text(postings_filtered).collect()


In [ ]:
_

In [ ]:
_ = partition_postings_and_write_title(postings_title_filtered).collect()

In [41]:
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp_text'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)

In [42]:
super_posting_locs

defaultdict(list,
            {"informer's": [('0_000.bin', 0)],
             'deep-blue': [('0_000.bin', 426)],
             'kurfürstendamm': [('0_000.bin', 996)],
             "shipley's": [('0_000.bin', 2400)],
             'krokowa': [('0_000.bin', 2940)],
             'gramophone': [('0_000.bin', 3324)],
             'ingjald': [('0_000.bin', 24768)],
             'ekeby': [('0_000.bin', 25308)],
             'dobroslav': [('0_000.bin', 25692)],
             'дети': [('0_000.bin', 26622)],
             'schistosoma': [('0_000.bin', 27240)],
             'erk1': [('0_000.bin', 28482)],
             'ballbronze': [('0_000.bin', 29130)],
             '1player': [('0_000.bin', 29466)],
             'brodie-sangster': [('0_000.bin', 30678)],
             '2-4-2t': [('0_000.bin', 31146)],
             'haute-garonne': [('0_000.bin', 31962)],
             'gorai': [('0_000.bin', 40716)],
             'context-based': [('0_000.bin', 41262)],
             'jen-hao': [('0_000.bin', 41736)]

In [ ]:
super_posting_locs_title = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp_title'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs_title[k].extend(v)

In [ ]:
# super_posting_locs_anc

In [43]:
# Create inverted index instance
inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict
# write the global stats out
inverted.write_index('.', 'index2')

In [44]:
# upload to gs
index_src = "index.pkl"
index_dst = f'gs://{bucket_name}/indexses/{index_src}'
!gsutil cp $index_src $index_dst

Copying file://index.pkl [Content-Type=application/octet-stream]...
/ [1 files][ 18.4 MiB/ 18.4 MiB]                                                
Operation completed over 1 objects/18.4 MiB.                                     


In [45]:
inverted

In [ ]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

all_stopwords = english_stopwords.union(corpus_stopwords)
def tokenize(text):
    """
    This function aims in tokenize a text into a list of tokens. Moreover, it filter stopwords.
    
    Parameters:
    -----------
    text: string , represting the text to tokenize.    
    
    Returns:
    -----------
    list of tokens (e.g., list of tokens).
    """
    list_of_tokens =  [token.group() for token in RE_WORD.finditer(text.lower()) if token.group() not in all_stopwords]    
    return list_of_tokens



In [ ]:
# Create inverted index instance
inverted_title = InvertedIndex_title()
# Adding the posting locations dictionary to the inverted index
inverted_title.posting_locs = super_posting_locs_title
# Add the token - df dictionary to the inverted index
inverted_title.df = w2df_dict_title
# write the global stats out
inverted_title.write_index('.', 'index_title')



In [ ]:
index_src = "index_title.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp_title/{index_src}'
!gsutil cp $index_src $index_dst

In [49]:
inverted_title.path = "postings_gcp_title"

inverted_anc.path =  "postings_gcp_anc"

inverted.path = "postings_gcp_text"

In [49]:
TUPLE_SIZE = 6       
TF_MASK = 2 ** 16 - 1 # Masking the 16 low bits of an integer
from contextlib import closing
# a finction that get an index and a token and return the posting_list
def read_posting_list(inverted, w):
  with closing(MultiFileReader()) as reader:
    locs = inverted.posting_locs[w]
    b = reader.read(locs, inverted.df[w] * TUPLE_SIZE)
    posting_list = []
    for i in range(inverted.df[w]):
      doc_id = int.from_bytes(b[i*TUPLE_SIZE:i*TUPLE_SIZE+4], 'big')
      tf = int.from_bytes(b[i*TUPLE_SIZE+4:(i+1)*TUPLE_SIZE], 'big')
      posting_list.append((doc_id, tf))
    return posting_list


In [1]:
from google.cloud import storage
TUPLE_SIZE = 6       
TF_MASK = 2 ** 16 - 1  
BLOCK_SIZE = 1999998


def post_index(word, index):
    client = storage.Client()
    bucket = client.bucket("316431873")
    b = []
    posting_list = [] 
    df = index.df[word]
    path = index.path
    with closing(MultiFileReader()) as reader:
        locs = index.posting_locs[word]
        n_bytes = index.df[word] * TUPLE_SIZE
#         b = reader.read(locs, n_bytes)
        for f_name, offset in locs:
                post_before = ""
                blob = bucket.blob(f"{path}/{f_name}")
                f = blob.open('rb')
#                 print(type(f))
                f.seek(offset)
                lst = [n_bytes , BLOCK_SIZE - offset]
                if lst[0] <= (BLOCK_SIZE - offset):
                    n_read = n_bytes
                else: 
                    n_read = BLOCK_SIZE - offset

                b.append(f.read(n_read))
                n_bytes -= n_read
                post_before = b''.join(b)
                for i in range(df):
                    doc_id = int.from_bytes(post_before[i*TUPLE_SIZE:i*TUPLE_SIZE+4], 'big')
                    tf = int.from_bytes(post_before[i*TUPLE_SIZE+4:(i+1)*TUPLE_SIZE], 'big')
                    posting_list.append((doc_id, tf))
        return posting_list




In [ ]:
doc_anchor_text_all

In [ ]:
doc_text_title= parquetFile.select("text", "id","title","text").rdd




In [ ]:
id_title_dict = dict(doc_title_pairs.map(lambda y: (y[1], y[0])).collect())

In [ ]:
output = open('id_title_dict.pkl', 'wb')
pickle.dump(id_title_dict, output)
output.close()

In [ ]:
output = open('DL.pkl', 'wb')
pickle.dump(dictle, output)
output.close()

In [52]:
inverted.df

{"army's": 18789,
 'silver': 204158,
 'behrman': 455,
 'rachmaninoff': 1552,
 'servitude': 2918,
 'northrup': 781,
 'strobl': 749,
 'dynamos': 1458,
 'kos': 2853,
 'am-fm': 294,
 '8481': 127,
 'storleer': 199,
 '1909-1994': 52,
 "yung's": 101,
 'jidaigeki': 643,
 'tiefe': 154,
 'field-programmable': 204,
 'prome': 339,
 'co-organizes': 68,
 'craske': 111,
 'petnjica': 69,
 'd-co': 141,
 'vasilievsky': 116,
 'corgo': 53,
 'darpa': 1611,
 'abstentions': 2390,
 'berléand': 116,
 'snakehead': 355,
 'waimate': 316,
 'interdependent': 1430,
 'barsby': 92,
 'lovebug': 93,
 'diffract': 110,
 'clapboard-sided': 101,
 'f5000': 144,
 "zidane's": 70,
 'grodzisko': 79,
 'fruit-tree': 57,
 "1893's": 51,
 'seven-acre': 118,
 'adores': 877,
 'woodcote': 317,
 'nona': 1733,
 'quick-drying': 64,
 'خالد': 112,
 'jixian': 126,
 'gayson': 61,
 'regenerate': 2826,
 '18-19': 782,
 'doreen': 3213,
 'brahmas': 195,
 'oligochaetes': 95,
 'pavol': 1131,
 'corsham': 437,
 'coolly': 661,
 'kersaint': 85,
 'quartus